# Assignment 2 for AIops course
SoSe 2023, presented by Aris Dressino  
Introduction is dedicated to exploratory data analysis and model compiling  
Review the causes and description in the initial part of every section  
See pdf appendix for a summary  

## Load data and set up environment

In [59]:
# Necessaries libraries
import pandas as pd
import json
import os
import datetime
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint;

In [2]:
# Load the ESB datase
esb = pd.read_csv('assignment2-data/esb.csv')

# Set the directory where the CSV files are located
dir_path = 'assignment2-data/host'
# Get a list of all the CSV files in the directory
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]
# Read each CSV file into a separate DataFrame and append it to a list
dfs = []
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(dir_path, csv_file))
    dfs.append(df)
# Combine all the DataFrames into a single DataFrame
hosts = pd.concat(dfs, ignore_index=True)

# Set the directory where the CSV files are located
dir_path = 'assignment2-data/trace'
# Get a list of all the CSV files in the directory
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]
# Read each CSV file into a separate DataFrame and append it to a list
dfs = []
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(dir_path, csv_file))
    dfs.append(df)
# Combine all the DataFrames into a single DataFrame
traces = pd.concat(dfs, ignore_index=True)

# Load the JSON file into a Python object
with open('failures.json', 'r') as f:
   failures = json.load(f)

## Exploratory Data Analysis - ESBs, hosts and traces

In [3]:
# Describe esb df
print(esb.describe(include="all"))
print(esb.count())
esb.head(5)

#720 entries

       serviceName     startTime    avg_time         num  succee_num  \
count          720  7.200000e+02  720.000000  720.000000  720.000000   
unique           1           NaN         NaN         NaN         NaN   
top        osb_001           NaN         NaN         NaN         NaN   
freq           720           NaN         NaN         NaN         NaN   
mean           NaN  1.590790e+12    0.845098  479.983333  467.445833   
std            NaN  1.247942e+07    0.953784   99.138401  122.559763   
min            NaN  1.590768e+12    0.000000   70.000000    0.000000   
25%            NaN  1.590779e+12    0.603650  429.750000  418.750000   
50%            NaN  1.590790e+12    0.637000  493.000000  488.500000   
75%            NaN  1.590800e+12    0.688975  550.250000  548.250000   
max            NaN  1.590811e+12    9.848100  652.000000  652.000000   

        succee_rate  
count    720.000000  
unique          NaN  
top             NaN  
freq            NaN  
mean       0.974795  
std

,serviceName,startTime,avg_time,num,succee_num,succee_rate
0,osb_001,1590768000000,0.6214,361,361,1.0000
1,osb_001,1590768060000,0.5861,343,343,1.0000
2,osb_001,1590768120000,5.0125,126,122,0.9683
3,osb_001,1590768180000,9.3487,70,70,1.0000
4,osb_001,1590768240000,1.4756,375,375,1.0000


In [4]:
# Describe hosts df
print(hosts.describe(include="all"))
print(hosts.count())
hosts.head(5)

#1,145,828 entries

              itemid          name     bomc_id     timestamp         value  \
count   1.145828e+06       1145828     1141228  1.145828e+06  1.145828e+06   
unique           NaN           128         128           NaN           NaN   
top              NaN  Disk_io_util  ZJ-001-072           NaN           NaN   
freq             NaN         43846       33961           NaN           NaN   
mean    7.824831e+14           NaN         NaN  1.590790e+12  5.540306e+07   
std     4.125572e+14           NaN         NaN  1.240014e+07  7.547781e+08   
min     4.871784e+07           NaN         NaN  1.590768e+12  0.000000e+00   
25%     1.000000e+15           NaN         NaN  1.590779e+12  0.000000e+00   
50%     1.000000e+15           NaN         NaN  1.590790e+12  5.800000e-01   
75%     1.000000e+15           NaN         NaN  1.590801e+12  2.362000e+01   
max     1.000000e+15           NaN         NaN  1.590811e+12  1.371952e+10   

        cmdb_id  
count   1145828  
unique       57  
top      

,itemid,name,bomc_id,timestamp,value,cmdb_id
0,999999996381288,container_cpu_used,ZJ-004-059,1590768012000,0.0,docker_006
1,999999996381284,container_mem_used,ZJ-004-060,1590768012000,93.0,docker_006
2,999999996381286,container_fgc,ZJ-004-103,1590768012000,0.0,docker_006
3,999999996381277,container_thread_idle,ZJ-004-063,1590768012000,0.0,docker_006
4,999999996381275,container_thread_total,ZJ-004-064,1590768012000,0.0,docker_006


In [5]:
# Describe traces df
#print(traces.describe(include="all"))
print(traces.count())
traces.head(5)

#19,739,965 entries

callType       19739965
startTime      19739965
msgTime        19739965
elapsedTime    19739965
success        19739965
traceId        19739965
id             19739965
pid            19739965
cmdb_id        19739965
dsName         15636991
serviceName    10904018
dtype: int64


,callType,startTime,msgTime,elapsedTime,success,traceId,id,pid,cmdb_id,dsName,serviceName
0,JDBC,1590768000183,1590768000189,6.0,True,37bd717261295c467616,b212a17261295cb74404,f65a417261295cb64401,docker_007,db_003,NaN
1,JDBC,1590768000188,1590768000193,5.0,True,2ebb417261295c4b7578,c321d17261295cbc3597,1fe7c17261295cbc3594,docker_008,db_003,NaN
2,JDBC,1590768000192,1590768000252,60.0,True,37bd717261295c467616,9b86517261295cc04407,f65a417261295cb64401,docker_007,db_003,NaN
3,JDBC,1590768000197,1590768000252,55.0,True,2ebb417261295c4b7578,9211617261295cc53604,1fe7c17261295cbc3594,docker_008,db_003,NaN
4,JDBC,1590768000215,1590768000226,11.0,True,ecda617261295c4c8924,1c19f17261295cd73614,f83ad17261295cd73611,docker_008,db_003,NaN


## List of failures to identify and detect root cause

In [6]:
# Show JSON content of failures
print(failures)

[[2220, [['os_021', 'Sent_queue']]], [6960, [['docker_007', None]]], [12480, [['os_017', 'Recv_total'], ['os_017', 'Send_total']]], [16080, [['db_003', 'On_Off_State'], ['db_003', 'tnsping_result_time']]], [28800, [['docker_005', None]]]]


### Auxiliary functions for data processing
- 30.05.2020 has been selected as refernce day by analysis of assignment2-data

In [7]:
#Function to extract info of failure programmatically
def extract_failure_info(failures, index):
    failure = failures[index]
    timestamp = failure[0]
    host = failure[1][0][0]
    metric = failure[1][0][1]
    return timestamp, host, metric

In [8]:
# Create function to transform seconds after a specified time in linux timestamp
def add_seconds_to_timestamp(timestamp: int, seconds: int) -> int:
    dt = datetime.datetime.fromtimestamp(timestamp) + datetime.timedelta(seconds=seconds)
    return int(dt.timestamp())

# Create anomaly detection model
- autoencoder model with threshold based on max mae train loss
- external checkpoints for weights retrieval
- based on assignment-2 data for hardware constraint, risk of over-fitting should be taken into account
- take traces as input and preprocess them

In [9]:
# PREPROCESSING
# Select the columns to be used for the autoencoder
cols = ['elapsedTime', 'success', 'traceId', 'id', 'pid', 'cmdb_id', 'callType'] # , 'startTime', 'msgTime', 'dsName', 'serviceName'] removed because of NaN values

# Create a new DataFrame with selected columns
traces_selected = traces[cols]

# Drop any rows with missing values
traces_selected = traces_selected.dropna()

# Apply one-hot encoding to categorical columns
traces_encoded = pd.get_dummies(traces_selected, columns=['cmdb_id', 'callType'])

# Scale the numerical columns
num_cols = ['elapsedTime']
scaler = StandardScaler()
traces_encoded[num_cols] = scaler.fit_transform(traces_encoded[num_cols])

# Set traceId, id, and pid columns as the index
traces_encoded.set_index(['traceId', 'id', 'pid'], inplace=True)

# Display the preprocessed DataFrame
traces_encoded.head(1)

,,,elapsedTime,success,cmdb_id_docker_001,cmdb_id_docker_002,cmdb_id_docker_003,cmdb_id_docker_004,cmdb_id_docker_005,cmdb_id_docker_006,cmdb_id_docker_007,cmdb_id_docker_008,cmdb_id_os_021,cmdb_id_os_022,callType_CSF,callType_FlyRemote,callType_JDBC,callType_LOCAL,callType_OSB,callType_RemoteProcess
traceId,id,pid,,,,,,,,,,,,,,,,,,
37bd717261295c467616,b212a17261295cb74404,f65a417261295cb64401,-0.147682,True,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [10]:
# AUTO-ENCODER ANOMALIES DETECTION MODEL COMPLILING
# Select relevant features
X = traces_encoded

# Split the data into training and testing sets
train_size = int(0.8 * len(X))
train_X, test_X = X[:train_size], X[train_size:]

# define the early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2, verbose=1, mode='auto')
# define checkpoint for weights caching
checkpoint = ModelCheckpoint("autoencoder_weights.h5", save_best_only=True, verbose=1)

# Define the autoencoder model
input_dim = train_X.shape[1]
encoding_dim = 2
input_layer = keras.Input(shape=(input_dim,))
encoder_layer1 = keras.layers.Dense(16, activation='relu')(input_layer)
encoder_layer2 = keras.layers.Dense(8, activation='relu')(encoder_layer1)
encoder_layer3 = keras.layers.Dense(encoding_dim, activation='relu',
                       activity_regularizer=keras.regularizers.l2(0.001))(encoder_layer2)
decoder_layer1 = keras.layers.Dense(16, activation='relu')(encoder_layer3)
decoder_layer2 = keras.layers.Dense(input_dim, activation='linear')(decoder_layer1)
autoencoder = keras.Model(inputs=input_layer, outputs=decoder_layer2)

# Define the optimizer with gradient clipping
adam = keras.optimizers.Adam(clipvalue=0.5)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Load weights from last checkpoint
autoencoder.load_weights("autoencoder_weights.h5")

# Train the model -> 1 or 2 epochs because of hardware constraints and performance
history = autoencoder.fit(train_X, train_X, epochs=1, batch_size=32, validation_data=(test_X, test_X), callbacks=[earlystop, checkpoint])

# Evaluate the model on the test data when loading weights
#loss = autoencoder.evaluate(train_X, train_X, verbose=0)
#print('Loss: {:.4f}'.format(loss))

# Evaluate the model
train_preds = autoencoder.predict(train_X)
train_mae_loss = np.mean(np.abs(train_X - train_preds), axis=1)
threshold = np.max(train_mae_loss)
test_preds = autoencoder.predict(test_X)
test_mae_loss = np.mean(np.abs(test_X - test_preds), axis=1)
anomalies = test_X[test_mae_loss > threshold]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 15791972 samples, validate on 3947993 samples
15791808/15791972 [============================>.] - ETA: 0s - loss: 0.0136
Epoch 00001: val_loss improved from inf to 0.10325, saving model to autoencoder_weights.h5
15791972/15791972 [==============================] - 6686s 423us/sample - loss: 0.0136 - val_loss: 0.1033


In [11]:
# show anomalies on test set and structure
print(anomalies.count())
anomalies.head(5)

elapsedTime               18
success                   18
cmdb_id_docker_001        18
cmdb_id_docker_002        18
cmdb_id_docker_003        18
cmdb_id_docker_004        18
cmdb_id_docker_005        18
cmdb_id_docker_006        18
cmdb_id_docker_007        18
cmdb_id_docker_008        18
cmdb_id_os_021            18
cmdb_id_os_022            18
callType_CSF              18
callType_FlyRemote        18
callType_JDBC             18
callType_LOCAL            18
callType_OSB              18
callType_RemoteProcess    18
dtype: int64


,,,elapsedTime,success,cmdb_id_docker_001,cmdb_id_docker_002,cmdb_id_docker_003,cmdb_id_docker_004,cmdb_id_docker_005,cmdb_id_docker_006,cmdb_id_docker_007,cmdb_id_docker_008,cmdb_id_os_021,cmdb_id_os_022,callType_CSF,callType_FlyRemote,callType_JDBC,callType_LOCAL,callType_OSB,callType_RemoteProcess
traceId,id,pid,,,,,,,,,,,,,,,,,,
f359d1726195d45c9002,5c8121726195d4672510,f5b3e1726195d4659005,207.408316,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3e3dd1726195d8119008,b3e371726195d8192669,0a0b41726195d8179011,202.627571,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4b3d91726195dd5f7578,e78f51726195ddb52991,8ed971726195dd647581,200.043769,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
fb8e61726195e02f9014,39a431726195e0363085,fbfe21726195e0349017,197.016319,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
6b2b11726195e10d7712,8139d1726195e1173097,a89aa1726195e1157715,195.525664,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


## 1st failure
- [2220, [['os_021', 'Sent_queue']]]  

transformed UNIX timestamp is 1590770220000, Fri May 29 2020 16:37:00 GMT+0000  
We observe that 5 ESB calls are logged in a 40 seconds interval (with defined timestamp as median):  
- 37     osb_001  1590770220000    1.9882  227         227          1.0 
**ESB call 37** has exactly the same timestamp when the falure was registered with avg_time higher than average  
With an abnormal high value, we can extract the host call relative to the failure:  
- 80657  999999996430078  Sent_queue  ZJ-001-064  1590770279000  11242.0  os_021

Using the anomaly detection autoencoder model, the traces calls in a 10 second interval (with defined timestamp as median) were extracted.  
For sake of clarity, we calculated the lowest time diff with the failure timestamp and extracted the 30 closes traces to the failure.  
- 3ecd0172614b3bfe1364	be537172614b3bfe1365	None	4.841705 OSB	1590770220030	1590770221590	1560.0	True	os_021	NaN	osb_001	30

We can derive that **TRACE id 3ecd0172614b3bfe1364** is the root cause without parent id having 1560.0 of response time, even 5x time longer than the average.  


In [12]:
# Look for the first failure
print(failures[0])
timestamp0, host0, metric0 = extract_failure_info(failures, 0)
timestamp = add_seconds_to_timestamp(1590768000, timestamp0)
print(timestamp*1000)

# Get the Linux timestamp 20 seconds earlier
start_unixtime = timestamp*1000 - 240000
#print(start_unixtime)
# Get the Linux timestamp 20 seconds later
end_unixtime = timestamp*1000 + 240000
#print(end_unixtime)

# extract values from the esb dataframe within the time range
esb_values = esb[(esb['startTime'] >= start_unixtime) & (esb['startTime'] <= end_unixtime)] # & (esb['succee_rate'] < 1.0)]
# extract values from the hosts dataframe within the time range by selected host and metric
hosts_values = hosts[(hosts['timestamp'] >= start_unixtime) & (hosts['timestamp'] <= end_unixtime) & (hosts['cmdb_id'] == host0) & (hosts['name'] == metric0)]
# extract values from the traces dataframe within the reduced time range (5 seconds) and by selected host
traces_values = traces[(traces['startTime'] >= start_unixtime+90000) & (traces['startTime'] <= end_unixtime-90000)] # & (traces['cmdb_id'] == host0)]
# Set traceId, id, and pid columns as the index
traces_values.set_index(['traceId', 'id', 'pid'], inplace=True)

[2220, [['os_021', 'Sent_queue']]]
1590770220000


In [13]:
#shows filtered ESBs and hosts values
print(esb_values)
print(hosts_values)
print(traces_values.count())

   serviceName      startTime  avg_time  num  succee_num  succee_rate
33     osb_001  1590769980000    0.6822  493         493          1.0
34     osb_001  1590770040000    0.5672  477         477          1.0
35     osb_001  1590770100000    0.6239  495         495          1.0
36     osb_001  1590770160000    0.6707  502         502          1.0
37     osb_001  1590770220000    1.9882  227         227          1.0
38     osb_001  1590770280000    2.3378  200         200          1.0
39     osb_001  1590770340000    2.2905  200         200          1.0
40     osb_001  1590770400000    2.2886  198         198          1.0
41     osb_001  1590770460000    0.6864  508         508          1.0
                itemid        name     bomc_id      timestamp    value cmdb_id
80657  999999996430078  Sent_queue  ZJ-001-064  1590770279000  11242.0  os_021
callType       87903
startTime      87903
msgTime        87903
elapsedTime    87903
success        87903
cmdb_id        87903
dsName         6

In [14]:
# FILTERED TRACES PREOPROCESSING
# Select the columns to be used for the autoencoder
cols = ['elapsedTime', 'success','cmdb_id', 'callType']

# Create a new DataFrame with selected columns
traces_selected = traces_values[cols]

# Drop any rows with missing values
traces_selected = traces_selected.dropna()

# Apply one-hot encoding to categorical columns
traces_encoded = pd.get_dummies(traces_selected, columns=['cmdb_id', 'callType'])

# Scale the numerical columns
num_cols = ['elapsedTime']
scaler = StandardScaler()
traces_encoded[num_cols] = scaler.fit_transform(traces_encoded[num_cols])

In [15]:
# AUTOENCODER PREDICTIONS
# Detect anomalies in selected traces interval
predictions = autoencoder.predict(traces_encoded)
# Set the threshold to a lower value than the default (e.g., 0.1 standard deviations above the mean), if needed
threshold = np.mean(train_mae_loss) #+ 0.1 * np.std(train_mae_loss)
pred_mae_loss = np.mean(np.abs(traces_encoded - predictions), axis=1)
anomalies = traces_encoded[pred_mae_loss > threshold]

In [16]:
# Get a list of all column names
cols_an = anomalies.columns.tolist()
cols_an.extend(['startTime', 'msgTime'])

# merge traces_df and traces_values based on traceId, id, and pid
time_anomalies = anomalies.join(traces_values, how='left', lsuffix='_x', rsuffix='_y')

#show anomalies per relevant host
time_anomalies = time_anomalies[time_anomalies['cmdb_id_os_021'] > 0]

In [17]:
# Extract 30 closest classified anomalies to target timestamp
target_timestamp = timestamp*1000
n_closest = 30

# Calculate the absolute difference between each timestamp and the target timestamp
time_anomalies['diff'] = abs(time_anomalies['startTime'] - target_timestamp)

# Sort the dataframe by the difference and select the top n_closest entries
closest_entries = time_anomalies.sort_values('diff').head(n_closest)
#Head only for readibility
closest_entries.head(5)

,,,elapsedTime_x,success_x,cmdb_id_docker_001,cmdb_id_docker_002,cmdb_id_docker_003,cmdb_id_docker_004,cmdb_id_docker_005,cmdb_id_docker_006,cmdb_id_docker_007,cmdb_id_docker_008,...,callType_RemoteProcess,callType,startTime,msgTime,elapsedTime_y,success_y,cmdb_id,dsName,serviceName,diff
traceId,id,pid,,,,,,,,,,,,,,,,,,,,,
3ecd0172614b3bfe1364,be537172614b3bfe1365,None,5.079014,True,0,0,0,0,0,0,0,0,...,0,OSB,1590770220030,1590770221590,1560.0,True,os_021,NaN,osb_001,30
5b876172614b3c051328,d92f3172614b3c051329,None,1.643357,True,0,0,0,0,0,0,0,0,...,0,OSB,1590770220037,1590770220576,539.0,True,os_021,NaN,osb_001,37
89ca9172614b3c051330,5a24e172614b3c051331,None,1.639992,True,0,0,0,0,0,0,0,0,...,0,OSB,1590770220037,1590770220576,538.0,True,os_021,NaN,osb_001,37
8285e172614b3c062674,0b987172614b3c062675,None,1.488567,True,0,0,0,0,0,0,0,0,...,0,OSB,1590770220038,1590770220531,493.0,True,os_021,NaN,osb_001,38
1fe1e172614b3c072676,c599b172614b3c072677,None,3.632067,True,0,0,0,0,0,0,0,0,...,0,OSB,1590770220039,1590770221169,1130.0,True,os_021,NaN,osb_001,39


## 2nd failure
- [6960, [['docker_007', None]]]  
transformed UNIX timestamp is 1590774960000, Fri May 29 2020 17:56:00 GMT+0000   
We observe that 5 ESB calls are logged in a 40 seconds interval (with defined timestamp as median):  
- 116     osb_001  1590774960000    0.7555  574         574       1.0000
**ESB call 116** has exactly the same timestamp when the falure was registered  
we can extract 6 the host calls relative to the failure's timestamp, showing the initialization of a docker container:
- 7340	999999996381309	container_fgc	ZJ-004-103	1590774928000	0.0	docker_007
- 7341	999999996381299	container_thread_running	ZJ-004-062	1590774928000	0.0	docker_007
- 7342	999999996381311	container_cpu_used	ZJ-004-059	1590774928000	21.0	docker_007
- 7343	999999996381307	container_mem_used	ZJ-004-060	1590774928000	60.0	docker_007
- 7407	999999996381298	container_thread_total	ZJ-004-064	1590774993000	0.0	docker_007
- 7408	999999996381310	container_fgct	ZJ-004-104	1590774993000	0.0	docker_007

**HOST call 7343**, the last one before the analyzed timestamp, shows a very high value of 60.0 for container memory used, probably definining the reason of the failure.  
Using the anomaly detection autoencoder model, the traces calls in a 10 second interval (with defined timestamp as median) were extracted.  
For sake of clarity, we calculated the lowest time diff with the failure timestamp and extracted the 30 closes traces to the failure.  
- 6b7d717261938fa16572	dc5fe17261938ffa5621	e2d0a17261938ff56930	-0.108560	RemoteProcess	1590774960122	1590774960145	23.0	True	docker_007	NaN	csf_004	122
- c5f8517261938fa06450	464d517261938ffc5623	73a4517261938ff96932	-0.099163	RemoteProcess	1590774960124	1590774960149	25.0	True	docker_007	NaN	csf_004	124

We can derive that **TRACE id 6b7d717261938fa16572** is the called containerized service failing and leading to a sucessive RemoteProcess log, as expected, the raise in memory used, lead to the failure of the docker process.  


In [18]:
# Look for the second failure
print(failures[1])
timestamp1, host1, metric1 = extract_failure_info(failures, 1)
timestamp = add_seconds_to_timestamp(1590768000, timestamp1)
print(timestamp*1000)

# Get the Linux timestamp 20 seconds earlier
start_unixtime = timestamp*1000 - 120000
#print(start_unixtime)
# Get the Linux timestamp 20 seconds later
end_unixtime = timestamp*1000 + 120000
#print(end_unixtime)

# extract values from the esb dataframe within the time range
esb_values = esb[(esb['startTime'] >= start_unixtime) & (esb['startTime'] <= end_unixtime)] # & (esb['succee_rate'] < 1.0)]
# extract values from the hosts dataframe within the time range by selected host
hosts_values = hosts[(hosts['timestamp'] >= start_unixtime) & (hosts['timestamp'] <= end_unixtime) & (hosts['cmdb_id'] == host1)]
# extract values from the traces dataframe within the reduced time range (5 seconds) and by selected host
traces_values = traces[(traces['startTime'] >= start_unixtime+90000) & (traces['startTime'] <= end_unixtime-90000)] # & (traces['cmdb_id'] == host0)]
# Set traceId, id, and pid columns as the index
traces_values.set_index(['traceId', 'id', 'pid'], inplace=True)

[6960, [['docker_007', None]]]
1590774960000


In [19]:
#shows filtered ESBs and hosts values
print(esb_values)
print(traces_values.count())
hosts_values.head(20)

    serviceName      startTime  avg_time  num  succee_num  succee_rate
114     osb_001  1590774840000    0.6557  607         607       1.0000
115     osb_001  1590774900000    0.6469  567         567       1.0000
116     osb_001  1590774960000    0.7555  574         574       1.0000
117     osb_001  1590775020000    0.8087  594         594       1.0000
118     osb_001  1590775080000    2.1664  371         370       0.9973
callType       31898
startTime      31898
msgTime        31898
elapsedTime    31898
success        31898
cmdb_id        31898
dsName         25301
serviceName    17593
dtype: int64


,itemid,name,bomc_id,timestamp,value,cmdb_id
7263,999999996381298,container_thread_total,ZJ-004-064,1590774873000,0.0,docker_007
7264,999999996381310,container_fgct,ZJ-004-104,1590774873000,0.0,docker_007
7265,999999996381300,container_thread_idle,ZJ-004-063,1590774873000,0.0,docker_007
7266,999999996353969,container_thread_used_pct,ZJ-004-061,1590774873000,0.0,docker_007
7267,999999996381301,container_session_used,ZJ-004-058,1590774873000,0.0,docker_007
7268,999999996381309,container_fgc,ZJ-004-103,1590774873000,0.0,docker_007
7269,999999996381299,container_thread_running,ZJ-004-062,1590774873000,0.0,docker_007
7270,999999996381311,container_cpu_used,ZJ-004-059,1590774873000,16.0,docker_007
7271,999999996381307,container_mem_used,ZJ-004-060,1590774873000,60.0,docker_007
7335,999999996381298,container_thread_total,ZJ-004-064,1590774928000,0.0,docker_007


In [20]:
# FILTERED TRACES PREOPROCESSING
# Select the columns to be used for the autoencoder
cols = ['elapsedTime', 'success','cmdb_id', 'callType']

# Create a new DataFrame with selected columns
traces_selected = traces_values[cols]

# Drop any rows with missing values
traces_selected = traces_selected.dropna()

# Apply one-hot encoding to categorical columns
traces_encoded = pd.get_dummies(traces_selected, columns=['cmdb_id', 'callType'])

# Scale the numerical columns
num_cols = ['elapsedTime']
scaler = StandardScaler()
traces_encoded[num_cols] = scaler.fit_transform(traces_encoded[num_cols])

In [21]:
# AUTOENCODER PREDICTIONS
# Detect anomalies in selected traces interval
predictions = autoencoder.predict(traces_encoded)
# Set the threshold to a lower value than the default (e.g., 0.1 standard deviations above the mean), if needed
threshold = np.mean(train_mae_loss) #+ 0.1 * np.std(train_mae_loss)
pred_mae_loss = np.mean(np.abs(traces_encoded - predictions), axis=1)
anomalies = traces_encoded[pred_mae_loss > threshold]

In [22]:
# Get a list of all column names
cols_an = anomalies.columns.tolist()
cols_an.extend(['startTime', 'msgTime'])

# merge traces_df and traces_values based on traceId, id, and pid
time_anomalies = anomalies.join(traces_values, how='left', lsuffix='_x', rsuffix='_y')

#show anomalies per relevant host
time_anomalies = time_anomalies[time_anomalies['cmdb_id_docker_007'] > 0]

In [23]:
# Extract 30 closest classified anomalies to target timestamp
target_timestamp = timestamp*1000
n_closest = 30

# Calculate the absolute difference between each timestamp and the target timestamp
time_anomalies['diff'] = abs(time_anomalies['startTime'] - target_timestamp)

# Sort the dataframe by the difference and select the top n_closest entries
closest_entries = time_anomalies.sort_values('diff').head(n_closest)
#Head only for readibility
closest_entries.head(5)

elapsedTime_x  \
traceId              id                   pid                                   
c5f8517261938fa06450 aacb717261938fdb5602 d4f7017261938fd76926      -0.085067   
                     107ea17261938fe25606 aacb717261938fdb5602      -0.174341   
                     c006817261938fe35609 107ea17261938fe25606      -0.207231   
                     800fc17261938fe85612 107ea17261938fe25606      -0.202532   
                     2f98f17261938fec5614 aacb717261938fdb5602      -0.197834   

                                                                success_x  \
traceId              id                   pid                               
c5f8517261938fa06450 aacb717261938fdb5602 d4f7017261938fd76926       True   
                     107ea17261938fe25606 aacb717261938fdb5602       True   
                     c006817261938fe35609 107ea17261938fe25606       True   
                     800fc17261938fe85612 107ea17261938fe25606       True   
                     2f98f17261938fec5614 aacb717261938fdb5602       True   

                                                                cmdb_id_docker_001  \
traceId              id                   pid                                        
c5f8517261938fa06450 aacb717261938fdb5602 d4f7017261938fd76926                   0   
                     107ea17261938fe25606 aacb717261938fdb5602                   0   
                     c006817261938fe35609 107ea17261938fe25606                   0   
                     800fc17261938fe85612 107ea17261938fe25606                   0   
                     2f98f17261938fec5614 aacb717261938fdb5602                   0   

                                                                cmdb_id_docker_002  \
traceId              id                   pid                                        
c5f8517261938fa06450 aacb717261938fdb5602 d4f7017261938fd76926                   0   
                     107ea17261938fe25606 aacb717261938fdb5602                   0   
                     c006817261938fe35609 107ea17261938fe25606                   0   
                     800fc17261938fe85612 107ea17261938fe25606                   0   
                     2f98f17261938fec5614 aacb717261938fdb5602                   0   

                                                                cmdb_id_docker_003  \
traceId              id                   pid                                        
c5f8517261938fa06450 aacb717261938fdb5602 d4f7017261938fd76926                   0   
                     107ea17261938fe25606 aacb717261938fdb5602                   0   
                     c006817261938fe35609 107ea17261938fe25606                   0   
                     800fc17261938fe85612 107ea17261938fe25606                   0   
                     2f98f17261938fec5614 aacb717261938fdb5602                   0   

                                                                cmdb_id_docker_004  \
traceId              id                   pid                                        
c5f8517261938fa06450 aacb717261938fdb5602 d4f7017261938fd76926                   0   
                     107ea17261938fe25606 aacb717261938fdb5602                   0   
                     c006817261938fe35609 107ea17261938fe25606                   0   
                     800fc17261938fe85612 107ea17261938fe25606                   0   
                     2f98f17261938fec5614 aacb717261938fdb5602                   0   

                                                                cmdb_id_docker_005  \
traceId              id                   pid                                        
c5f8517261938fa06450 aacb717261938fdb5602 d4f7017261938fd76926                   0   
                     107ea17261938fe25606 aacb717261938fdb5602                   0   
                     c006817261938fe35609 107ea17261938fe25606                   0   
                     800fc17261938fe85612 107ea17261938fe25606                   0   
                     2f98f17

## 3rd failure
- [12480, [['os_017', 'Recv_total'], ['os_017', 'Send_total']]]
transformed UNIX timestamp is 1590780480000, Fri May 29 2020 19:28:00 GMT+0000   
We observe that 5 ESB calls are logged in a 40 seconds interval (with defined timestamp as median):  
- 208     osb_001  1590780480000    7.9690   83          83          1.0
**ESB call 208** has exactly the same timestamp when the falure was registered  
After, we can extract 4 the host calls relative to the failure's timestamp:
- 211635  999999996487800  Send_total  ZJ-001-074  1590780480000    0.201373   
- 211639  999999996487801  Recv_total  ZJ-001-073  1590780481000  203.077040   
- 212457  999999996487800  Send_total  ZJ-001-074  1590780541000    0.049255   
- 212464  999999996487801  Recv_total  ZJ-001-073  1590780542000   51.943858

**HOST call 12457** shows an abnormal low value for the metric Sends_total, probably definining the reason of the failure.  

Using the anomaly detection autoencoder model, the traces calls in a 10 second interval (with defined timestamp as median) were extracted.  
For sake of clarity, we calculated the lowest time diff with the failure timestamp and extracted the 60 closes traces to the failure.  
- c478117261e7ca505848	7278c17261e7ca9a7273	4603317261e7ca857264	-0.218956   LOCAL	1590780480154	 1590780480160	6.0	True	docker_005	db_003	local_method_011	154
- c478117261e7ca505848  6e1ce17261e7ca9a7276	7278c17261e7ca9a7273	-0.220327   JDBC	1590780480154	1590780480156	2.0	True	docker_005	db_003	NaN	154
- 4c90617261e7ca572726	8275417261e7ca9c7281	20b4317261e7ca857262	-0.218956	LOCAL	1590780480156	1590780480162	6.0	True	docker_005	db_003	local_method_011	156
- 4c90617261e7ca572726  5ef1f17261e7ca9c7284	8275417261e7ca9c7281	-0.219984	JDBC	1590780480156	1590780480159	3.0	True	docker_005	db_003	NaN	156
- c478117261e7ca505848	0d68b17261e7ca9d7287	7278c17261e7ca9a7273	-0.219984	JDBC	1590780480157	1590780480160	3.0	True	docker_005	db_003	NaN	157
- 4c90617261e7ca572726	1558917261e7caa07290	8275417261e7ca9c7281	-0.220327	JDBC	1590780480160	1590780480162	2.0	True	docker_005	db_003	NaN	160

With 5 consecutive calls to docker_005, we can derive that **TRACE id c478117261e7ca505848** is the root cause having 6.0 seconds duration and triggering a repetition of the logs after his call, showing a failure in the first attempt.  

In [24]:
# Look for the third failure
print(failures[2])
timestamp = add_seconds_to_timestamp(1590768000, 12480)
print(timestamp*1000)

# Get the Linux timestamp 20 seconds earlier
start_unixtime = timestamp*1000 - 120000
#print(start_unixtime)
# Get the Linux timestamp 20 seconds later
end_unixtime = timestamp*1000 + 120000
#print(end_unixtime)

# extract values from the esb dataframe within the time range
esb_values = esb[(esb['startTime'] >= start_unixtime) & (esb['startTime'] <= end_unixtime)] # & (esb['succee_rate'] < 1.0)]
# extract values from the hosts dataframe within the time range by selected host and metric
hosts_values = hosts[(hosts['timestamp'] >= start_unixtime) & (hosts['timestamp'] <= end_unixtime) & (hosts['cmdb_id'] == 'os_017') & ((hosts['name'] == 'Recv_total') | (hosts['name'] == 'Send_total'))]
# extract values from the traces dataframe within the reduced time range (5 seconds) and by selected host
traces_values = traces[(traces['startTime'] >= start_unixtime+90000) & (traces['startTime'] <= end_unixtime-90000)] # & (traces['cmdb_id'] == host0)]
# Set traceId, id, and pid columns as the index
traces_values.set_index(['traceId', 'id', 'pid'], inplace=True)

[12480, [['os_017', 'Recv_total'], ['os_017', 'Send_total']]]
1590780480000


In [25]:
#shows filtered ESBs and hosts values
print(esb_values)
print(hosts_values)
print(traces_values.count())

    serviceName      startTime  avg_time  num  succee_num  succee_rate
206     osb_001  1590780360000    0.5808  436         436          1.0
207     osb_001  1590780420000    0.5621  391         391          1.0
208     osb_001  1590780480000    7.9690   83          83          1.0
209     osb_001  1590780540000    7.3649   91          91          1.0
210     osb_001  1590780600000    8.4769   80          80          1.0
                 itemid        name     bomc_id      timestamp       value  \
210028  999999996487800  Send_total  ZJ-001-074  1590780360000    0.210310   
210033  999999996487801  Recv_total  ZJ-001-073  1590780361000  214.408331   
210831  999999996487800  Send_total  ZJ-001-074  1590780420000    0.226036   
210836  999999996487801  Recv_total  ZJ-001-073  1590780421000  231.597301   
211635  999999996487800  Send_total  ZJ-001-074  1590780480000    0.201373   
211639  999999996487801  Recv_total  ZJ-001-073  1590780481000  203.077040   
212457  999999996487800  Sen

In [43]:
# FILTERED TRACES PREOPROCESSING
# Select the columns to be used for the autoencoder
cols = ['elapsedTime', 'success','cmdb_id', 'callType']

# Create a new DataFrame with selected columns
traces_selected = traces_values[cols]

# Drop any rows with missing values
traces_selected = traces_selected.dropna()

# Apply one-hot encoding to categorical columns
traces_encoded = pd.get_dummies(traces_selected, columns=['cmdb_id', 'callType'])

# Scale the numerical columns
num_cols = ['elapsedTime']
scaler = StandardScaler()
traces_encoded[num_cols] = scaler.fit_transform(traces_encoded[num_cols])

In [44]:
# AUTOENCODER PREDICTIONS
# Detect anomalies in selected traces interval
predictions = autoencoder.predict(traces_encoded)
# Set the threshold to a lower value than the default (e.g., 0.1 standard deviations above the mean), if needed
threshold = np.mean(train_mae_loss) #+ 0.1 * np.std(train_mae_loss)
pred_mae_loss = np.mean(np.abs(traces_encoded - predictions), axis=1)
anomalies = traces_encoded[pred_mae_loss > threshold]

In [45]:
# Get a list of all column names
cols_an = anomalies.columns.tolist()
cols_an.extend(['startTime', 'msgTime'])

# merge traces_df and traces_values based on traceId, id, and pid
time_anomalies = anomalies.join(traces_values, how='left', lsuffix='_x', rsuffix='_y')

#show anomalies per relevant host -> os_017 not included in traces
#time_anomalies = time_anomalies[time_anomalies['cmdb_id_os_017'] > 0]

In [46]:
# Extract 60 closest classified anomalies to target timestamp
target_timestamp = timestamp*1000
n_closest = 60

# Calculate the absolute difference between each timestamp and the target timestamp
time_anomalies['diff'] = abs(time_anomalies['startTime'] - target_timestamp)

# Sort the dataframe by the difference and select the top n_closest entries
closest_entries = time_anomalies.sort_values('diff').head(n_closest)
#Head only for readibility
closest_entries.head(5)

,,,elapsedTime_x,success_x,cmdb_id_docker_001,cmdb_id_docker_002,cmdb_id_docker_003,cmdb_id_docker_004,cmdb_id_docker_005,cmdb_id_docker_006,cmdb_id_docker_007,cmdb_id_docker_008,...,callType_RemoteProcess,callType,startTime,msgTime,elapsedTime_y,success_y,cmdb_id,dsName,serviceName,diff
traceId,id,pid,,,,,,,,,,,,,,,,,,,,,
43ee517262e0d01b9334,8a95317262e0d01b9335,None,0.530199,True,0,0,0,0,0,0,0,0,...,0,OSB,1590796800027,1590796800399,372.0,True,os_021,NaN,osb_001,27
d377317262e0d01c1686,6c60a17262e0d01c1687,None,0.485632,True,0,0,0,0,0,0,0,0,...,0,OSB,1590796800028,1590796800375,347.0,True,os_021,NaN,osb_001,28
738d617262e0d01f9338,a595e17262e0d01f9339,None,0.968736,True,0,0,0,0,0,0,0,0,...,0,OSB,1590796800031,1590796800649,618.0,True,os_021,NaN,osb_001,31
ada7017262e0d0209248,00e5f17262e0d0209249,None,2.414483,True,0,0,0,0,0,0,0,0,...,0,OSB,1590796800032,1590796801461,1429.0,True,os_021,NaN,osb_001,32
43ee517262e0d01b9334,724b817262e0d0249341,8a95317262e0d01b9335,0.460675,True,0,0,0,0,0,0,0,0,...,0,CSF,1590796800036,1590796800369,333.0,True,os_021,NaN,csf_001,36


## 4th failure
- [16080, [['db_003', 'On_Off_State'], ['db_003', 'tnsping_result_time']]]
transformed UNIX timestamp is 1590784080000, Fri May 29 2020 20:28:00 GMT+0000  
We observe that 3 ESB calls with 0 as success rate are logged in a 40 seconds interval (with defined timestamp as median):  
- 268     osb_001  1590784080000    0.4231  496         143       0.2883
- 269     osb_001  1590784140000    0.0000  465           0       0.0000
- 270     osb_001  1590784200000    0.0000  493           0       0.0000
**ESB call 268** has exactly the same timestamp when the falure was registered  
After, we can extract 2 the host calls relative to the failure's timestamp:
- 760203	999999998650977	tnsping_result_time	ZJ-002-031	1590784137000	99999.0	db_003
- 760653	999999998650977	tnsping_result_time	ZJ-002-031	1590784197000	99999.0	db_003

**HOST call 760203** shows an abnormal hugh value of 99999.0 for the metric tnsping_result_time, definining the reason of the failure.  

Using the anomaly detection autoencoder model, the traces calls in a 10 second interval (with defined timestamp as median) were extracted.  
For sake of clarity, we calculated the lowest time diff with the failure timestamp and extracted the 60 closes traces to the failure.  
- 5487e172621eb8944084	ec99f172621eb8cd9301	fec2a172621eb8b49289	-0.206633	LOCAL	1590784080077	1590784080087	10.0	True	docker_007	db_003	local_method_011	77
- 5487e172621eb8944084  81dcf172621eb8cd9304	ec99f172621eb8cd9301	-0.250095	JDBC	1590784080077	1590784080080	3.0	True	docker_007	db_003	NaN	77
- 14ac9172621eb89a5524	bfca6172621eb8ce9306	f8a8f172621eb8b49287	-0.212842	LOCAL	1590784080078	1590784080087	9.0	True	docker_007	db_003	local_method_011	78
- 14ac9172621eb89a5524  ca695172621eb8ce9309	bfca6172621eb8ce9306	-0.250095	JDBC	1590784080078	1590784080081	3.0	True	docker_007	db_003	NaN	78
- 14ac9172621eb89a5524  f59b2172621eb8d39315	bfca6172621eb8ce9306	-0.250095	JDBC	1590784080083	1590784080086	3.0	True	docker_007	db_003	NaN	83

With consecutive calls to db_003, we can derive that **TRACE id 5487e172621eb8944084** is the root cause having a generated error leading to multiple close call to the same service with similar duration  

In [47]:
# Look for the fourth failure
print(failures[3])
timestamp = add_seconds_to_timestamp(1590768000, 16080)
print(timestamp*1000)

# Get the Linux timestamp 20 seconds earlier
start_unixtime = timestamp*1000 - 120000
#print(start_unixtime)
# Get the Linux timestamp 37 seconds later
end_unixtime = timestamp*1000 + 120000
#print(end_unixtime)

# extract values from the esb dataframe within the time range
esb_values = esb[(esb['startTime'] >= start_unixtime) & (esb['startTime'] <= end_unixtime)] # & (esb['succee_rate'] < 1.0)]
# extract values from the hosts dataframe within the time range by selected host and metric
hosts_values = hosts[(hosts['timestamp'] >= start_unixtime) & (hosts['timestamp'] <= end_unixtime) & (hosts['cmdb_id'] == 'db_003') & ((hosts['name'] == 'On_Off_State') | (hosts['name'] == 'tnsping_result_time'))]
# extract values from the traces dataframe within the reduced time range (5 seconds) and by selected host
traces_values = traces[(traces['startTime'] >= start_unixtime+90000) & (traces['startTime'] <= end_unixtime-90000)] # & (traces['cmdb_id'] == host0)]
# Set traceId, id, and pid columns as the index
traces_values.set_index(['traceId', 'id', 'pid'], inplace=True)

[16080, [['db_003', 'On_Off_State'], ['db_003', 'tnsping_result_time']]]
1590784080000


In [48]:
#shows filtered ESBs and hosts values
print(esb_values)
print(traces_values.count())
hosts_values

    serviceName      startTime  avg_time  num  succee_num  succee_rate
266     osb_001  1590783960000    0.6005  462         462       1.0000
267     osb_001  1590784020000    0.6565  446         446       1.0000
268     osb_001  1590784080000    0.4231  496         143       0.2883
269     osb_001  1590784140000    0.0000  465           0       0.0000
270     osb_001  1590784200000    0.0000  493           0       0.0000
callType       15985
startTime      15985
msgTime        15985
elapsedTime    15985
success        15985
cmdb_id        15985
dsName         11115
serviceName     9890
dtype: int64


,itemid,name,bomc_id,timestamp,value,cmdb_id
759214,999999998650999,On_Off_State,ZJ-002-030,1590784003000,1.0,db_003
759328,999999998650977,tnsping_result_time,ZJ-002-031,1590784017000,0.0,db_003
759687,999999998650999,On_Off_State,ZJ-002-030,1590784063000,1.0,db_003
759804,999999998650977,tnsping_result_time,ZJ-002-031,1590784077000,0.0,db_003
760203,999999998650977,tnsping_result_time,ZJ-002-031,1590784137000,99999.0,db_003
760653,999999998650977,tnsping_result_time,ZJ-002-031,1590784197000,99999.0,db_003


In [49]:
# FILTERED TRACES PREOPROCESSING
# Select the columns to be used for the autoencoder
cols = ['elapsedTime', 'success','cmdb_id', 'callType']

# Create a new DataFrame with selected columns
traces_selected = traces_values[cols]

# Drop any rows with missing values
traces_selected = traces_selected.dropna()

# Apply one-hot encoding to categorical columns
traces_encoded = pd.get_dummies(traces_selected, columns=['cmdb_id', 'callType'])

# Scale the numerical columns
num_cols = ['elapsedTime']
scaler = StandardScaler()
traces_encoded[num_cols] = scaler.fit_transform(traces_encoded[num_cols])

In [50]:
# AUTOENCODER PREDICTIONS
# Detect anomalies in selected traces interval
predictions = autoencoder.predict(traces_encoded)
# Set the threshold to a lower value than the default (e.g., 0.1 standard deviations above the mean), if needed
threshold = np.mean(train_mae_loss) #+ 0.1 * np.std(train_mae_loss)
pred_mae_loss = np.mean(np.abs(traces_encoded - predictions), axis=1)
anomalies = traces_encoded[pred_mae_loss > threshold]

In [51]:
# Get a list of all column names
cols_an = anomalies.columns.tolist()
cols_an.extend(['startTime', 'msgTime'])

# merge traces_df and traces_values based on traceId, id, and pid
time_anomalies = anomalies.join(traces_values, how='left', lsuffix='_x', rsuffix='_y')

#show anomalies per relevant host
time_anomalies = time_anomalies[time_anomalies['dsName'] == 'db_003' ]

In [52]:
# Extract 30 closest classified anomalies to target timestamp
target_timestamp = timestamp*1000
n_closest = 30

# Calculate the absolute difference between each timestamp and the target timestamp
time_anomalies['diff'] = abs(time_anomalies['startTime'] - target_timestamp)

# Sort the dataframe by the difference and select the top n_closest entries
closest_entries = time_anomalies.sort_values('diff').head(n_closest)
#Head only for readibility
closest_entries.head(5)

elapsedTime_x  \
traceId              id                   pid                                   
5487e172621eb8944084 ec99f172621eb8cd9301 fec2a172621eb8b49289      -0.206633   
                     81dcf172621eb8cd9304 ec99f172621eb8cd9301      -0.250095   
14ac9172621eb89a5524 bfca6172621eb8ce9306 f8a8f172621eb8b49287      -0.212842   
                     ca695172621eb8ce9309 bfca6172621eb8ce9306      -0.250095   
                     f59b2172621eb8d39315 bfca6172621eb8ce9306      -0.250095   

                                                                success_x  \
traceId              id                   pid                               
5487e172621eb8944084 ec99f172621eb8cd9301 fec2a172621eb8b49289       True   
                     81dcf172621eb8cd9304 ec99f172621eb8cd9301       True   
14ac9172621eb89a5524 bfca6172621eb8ce9306 f8a8f172621eb8b49287       True   
                     ca695172621eb8ce9309 bfca6172621eb8ce9306       True   
                     f59b2172621eb8d39315 bfca6172621eb8ce9306       True   

                                                                cmdb_id_docker_001  \
traceId              id                   pid                                        
5487e172621eb8944084 ec99f172621eb8cd9301 fec2a172621eb8b49289                   0   
                     81dcf172621eb8cd9304 ec99f172621eb8cd9301                   0   
14ac9172621eb89a5524 bfca6172621eb8ce9306 f8a8f172621eb8b49287                   0   
                     ca695172621eb8ce9309 bfca6172621eb8ce9306                   0   
                     f59b2172621eb8d39315 bfca6172621eb8ce9306                   0   

                                                                cmdb_id_docker_002  \
traceId              id                   pid                                        
5487e172621eb8944084 ec99f172621eb8cd9301 fec2a172621eb8b49289                   0   
                     81dcf172621eb8cd9304 ec99f172621eb8cd9301                   0   
14ac9172621eb89a5524 bfca6172621eb8ce9306 f8a8f172621eb8b49287                   0   
                     ca695172621eb8ce9309 bfca6172621eb8ce9306                   0   
                     f59b2172621eb8d39315 bfca6172621eb8ce9306                   0   

                                                                cmdb_id_docker_003  \
traceId              id                   pid                                        
5487e172621eb8944084 ec99f172621eb8cd9301 fec2a172621eb8b49289                   0   
                     81dcf172621eb8cd9304 ec99f172621eb8cd9301                   0   
14ac9172621eb89a5524 bfca6172621eb8ce9306 f8a8f172621eb8b49287                   0   
                     ca695172621eb8ce9309 bfca6172621eb8ce9306                   0   
                     f59b2172621eb8d39315 bfca6172621eb8ce9306                   0   

                                                                cmdb_id_docker_004  \
traceId              id                   pid                                        
5487e172621eb8944084 ec99f172621eb8cd9301 fec2a172621eb8b49289                   0   
                     81dcf172621eb8cd9304 ec99f172621eb8cd9301                   0   
14ac9172621eb89a5524 bfca6172621eb8ce9306 f8a8f172621eb8b49287                   0   
                     ca695172621eb8ce9309 bfca6172621eb8ce9306                   0   
                     f59b2172621eb8d39315 bfca6172621eb8ce9306                   0   

                                                                cmdb_id_docker_005  \
traceId              id                   pid                                        
5487e172621eb8944084 ec99f172621eb8cd9301 fec2a172621eb8b49289                   0   
                     81dcf172621eb8cd9304 ec99f172621eb8cd9301                   0   
14ac9172621eb89a5524 bfca6172621eb8ce9306 f8a8f172621eb8b49287                   0   
                     ca695172621eb8ce9309 bfca6172621eb8ce9306                   0   
                     f59b217

## 5th failure
- 28800, [['docker_005', None]]]
transformed UNIX timestamp is 1590796800000, Sat May 30 2020 00:00:00 GMT+0000  
We observe that 6 ESB calls are logged in a 40 seconds interval (with defined timestamp as median):  
- 480     osb_001  1590796800000    3.0143  206         206          1.0
**ESB call 480** has exactly the same timestamp when the falure was registered with avg_time 6x usual duration.   
After, we can extract 20+ the host calls relative to the failure's timestamp:
- 31883	999999996381265	container_cpu_used	ZJ-004-059	1590796697000	17.0	docker_005
- 31884	999999996381253	container_thread_running	ZJ-004-062	1590796697000	0.0	docker_005
- 31885	999999996381261	container_mem_used	ZJ-004-060	1590796697000	57.0	docker_005

**HOST call 318856** triggers multiple identical processes trying to initialize container docker_005 after consuming dedicated CPU and memory.  

Using the anomaly detection autoencoder model, the traces calls in a 10 second interval (with defined timestamp as median) were extracted.  
For sake of clarity, we calculated the lowest time diff with the failure timestamp and extracted the 60 closes traces to the failure.  
- feb9a17262e0d9278646	2c9ac17262e0d9872150	b14ea17262e0d9848748	-0.086606	RemoteProcess	1590796802439	1590796802465	26.0	True	docker_005	NaN	csf_004	2439
- f3cc717262e0d9318650	77dea17262e0d9872148	7399a17262e0d9857289	-0.086606	RemoteProcess	1590796802439	1590796802465	26.0	True	docker_005	NaN	csf_004	2439

With consecutive calls to app2, we can derive that **TRACE id feb9a17262e0d9278646** is the root cause having a generating an identical requests in the same moment

In [53]:
# Look for the fifth failure
print(failures[4])
timestamp4, host4, metric4 = extract_failure_info(failures, 4)
timestamp = add_seconds_to_timestamp(1590768000, timestamp4)
print(timestamp*1000)

# Get the Linux timestamp 20 seconds earlier
start_unixtime = timestamp*1000 - 120000
#print(start_unixtime)
# Get the Linux timestamp 20 seconds later
end_unixtime = timestamp*1000 + 120000
#print(end_unixtime)

# extract values from the esb dataframe within the time range
esb_values = esb[(esb['startTime'] >= start_unixtime) & (esb['startTime'] <= end_unixtime)] # & (esb['succee_rate'] < 1.0)]
# extract values from the hosts dataframe within the time range by selected host and metric
hosts_values = hosts[(hosts['timestamp'] >= start_unixtime) & (hosts['timestamp'] <= end_unixtime) & (hosts['cmdb_id'] == host4)]
# extract values from the traces dataframe within the reduced time range (5 seconds) and by selected host
traces_values = traces[(traces['startTime'] >= start_unixtime+90000) & (traces['startTime'] <= end_unixtime-90000)] # & (traces['cmdb_id'] == host0)]
# Set traceId, id, and pid columns as the index
traces_values.set_index(['traceId', 'id', 'pid'], inplace=True)

[28800, [['docker_005', None]]]
1590796800000


In [54]:
#shows filtered ESBs and hosts values
print(esb_values)
print(traces_values.count())
hosts_values.head(5)

    serviceName      startTime  avg_time  num  succee_num  succee_rate
478     osb_001  1590796680000    0.6219  512         512          1.0
479     osb_001  1590796740000    0.5540  635         635          1.0
480     osb_001  1590796800000    3.0143  206         206          1.0
481     osb_001  1590796860000    3.0125  207         207          1.0
482     osb_001  1590796920000    2.9031  185         185          1.0
callType       17416
startTime      17416
msgTime        17416
elapsedTime    17416
success        17416
cmdb_id        17416
dsName         13786
serviceName     9618
dtype: int64


,itemid,name,bomc_id,timestamp,value,cmdb_id
31878,999999996381255,container_session_used,ZJ-004-058,1590796697000,0.0,docker_005
31879,999999996381263,container_fgc,ZJ-004-103,1590796697000,0.0,docker_005
31880,999999996353971,container_thread_used_pct,ZJ-004-061,1590796697000,0.0,docker_005
31881,999999996381252,container_thread_total,ZJ-004-064,1590796697000,0.0,docker_005
31882,999999996381264,container_fgct,ZJ-004-104,1590796697000,0.0,docker_005


In [55]:
# FILTERED TRACES PREOPROCESSING
# Select the columns to be used for the autoencoder
cols = ['elapsedTime', 'success','cmdb_id', 'callType']

# Create a new DataFrame with selected columns
traces_selected = traces_values[cols]

# Drop any rows with missing values
traces_selected = traces_selected.dropna()

# Apply one-hot encoding to categorical columns
traces_encoded = pd.get_dummies(traces_selected, columns=['cmdb_id', 'callType'])

# Scale the numerical columns
num_cols = ['elapsedTime']
scaler = StandardScaler()
traces_encoded[num_cols] = scaler.fit_transform(traces_encoded[num_cols])

In [56]:
# AUTOENCODER PREDICTIONS
# Detect anomalies in selected traces interval
predictions = autoencoder.predict(traces_encoded)
# Set the threshold to a lower value than the default (e.g., 0.1 standard deviations above the mean), if needed
threshold = np.mean(train_mae_loss) #+ 0.1 * np.std(train_mae_loss)
pred_mae_loss = np.mean(np.abs(traces_encoded - predictions), axis=1)
anomalies = traces_encoded[pred_mae_loss > threshold]

In [57]:
# Get a list of all column names
cols_an = anomalies.columns.tolist()
cols_an.extend(['startTime', 'msgTime'])

# merge traces_df and traces_values based on traceId, id, and pid
time_anomalies = anomalies.join(traces_values, how='left', lsuffix='_x', rsuffix='_y')

#show anomalies per relevant host trace for host docker_005 or parent process os_017
time_anomalies = time_anomalies[(time_anomalies['cmdb_id'] == 'docker_005')]

In [58]:
# Extract 30 closest classified anomalies to target timestamp
target_timestamp = timestamp*1000
n_closest = 30

# Calculate the absolute difference between each timestamp and the target timestamp
time_anomalies['diff'] = abs(time_anomalies['startTime'] - target_timestamp)

# Sort the dataframe by the difference and select the top n_closest entries
closest_entries = time_anomalies.sort_values('diff').head(n_closest)
#Head only for readibility
closest_entries.head(5)

elapsedTime_x  \
traceId              id                   pid                                   
1dba217262e0d0a08332 0d6fd17262e0d0e32063 1631a17262e0d0e07111      -0.074127   
                     53f7017262e0d0f72073 5db4217262e0d0f72070      -0.127607   
                     5db4217262e0d0f72070 0d6fd17262e0d0e32063      -0.120477   
                     1350f17262e0d0fb2076 5db4217262e0d0f72070      -0.127607   
                     bf3d017262e0d10b2078 6ac1d17262e0d1087115      -0.093736   

                                                                success_x  \
traceId              id                   pid                               
1dba217262e0d0a08332 0d6fd17262e0d0e32063 1631a17262e0d0e07111       True   
                     53f7017262e0d0f72073 5db4217262e0d0f72070       True   
                     5db4217262e0d0f72070 0d6fd17262e0d0e32063       True   
                     1350f17262e0d0fb2076 5db4217262e0d0f72070       True   
                     bf3d017262e0d10b2078 6ac1d17262e0d1087115       True   

                                                                cmdb_id_docker_001  \
traceId              id                   pid                                        
1dba217262e0d0a08332 0d6fd17262e0d0e32063 1631a17262e0d0e07111                   0   
                     53f7017262e0d0f72073 5db4217262e0d0f72070                   0   
                     5db4217262e0d0f72070 0d6fd17262e0d0e32063                   0   
                     1350f17262e0d0fb2076 5db4217262e0d0f72070                   0   
                     bf3d017262e0d10b2078 6ac1d17262e0d1087115                   0   

                                                                cmdb_id_docker_002  \
traceId              id                   pid                                        
1dba217262e0d0a08332 0d6fd17262e0d0e32063 1631a17262e0d0e07111                   0   
                     53f7017262e0d0f72073 5db4217262e0d0f72070                   0   
                     5db4217262e0d0f72070 0d6fd17262e0d0e32063                   0   
                     1350f17262e0d0fb2076 5db4217262e0d0f72070                   0   
                     bf3d017262e0d10b2078 6ac1d17262e0d1087115                   0   

                                                                cmdb_id_docker_003  \
traceId              id                   pid                                        
1dba217262e0d0a08332 0d6fd17262e0d0e32063 1631a17262e0d0e07111                   0   
                     53f7017262e0d0f72073 5db4217262e0d0f72070                   0   
                     5db4217262e0d0f72070 0d6fd17262e0d0e32063                   0   
                     1350f17262e0d0fb2076 5db4217262e0d0f72070                   0   
                     bf3d017262e0d10b2078 6ac1d17262e0d1087115                   0   

                                                                cmdb_id_docker_004  \
traceId              id                   pid                                        
1dba217262e0d0a08332 0d6fd17262e0d0e32063 1631a17262e0d0e07111                   0   
                     53f7017262e0d0f72073 5db4217262e0d0f72070                   0   
                     5db4217262e0d0f72070 0d6fd17262e0d0e32063                   0   
                     1350f17262e0d0fb2076 5db4217262e0d0f72070                   0   
                     bf3d017262e0d10b2078 6ac1d17262e0d1087115                   0   

                                                                cmdb_id_docker_005  \
traceId              id                   pid                                        
1dba217262e0d0a08332 0d6fd17262e0d0e32063 1631a17262e0d0e07111                   1   
                     53f7017262e0d0f72073 5db4217262e0d0f72070                   1   
                     5db4217262e0d0f72070 0d6fd17262e0d0e32063                   1   
                     1350f17262e0d0fb2076 5db4217262e0d0f72070                   1   
                     bf3d017